In [1]:
# define presentations of X and Y
endomap_alpha = {
    'generators':['a','b','c','d'],
    'relations':[
        (('a',5),('a',2)),
        (('a',2),('b',1)),
        (('a',3),('c',1)),
        (('d',2),('d',0))        
    ]
}

endomap_beta = {
    'generators':['p','q','z','m'],
    'relations':[
        (('p',1),('q',1)),
        (('p',2),('p',6)),
        (('z',4),('z',1)),
        (('m',2),('m',0))        
    ]
}


print(endomap_alpha)
print(endomap_beta)

{'generators': ['a', 'b', 'c', 'd'], 'relations': [(('a', 5), ('a', 2)), (('a', 2), ('b', 1)), (('a', 3), ('c', 1)), (('d', 2), ('d', 0))]}
{'generators': ['p', 'q', 'z', 'm'], 'relations': [(('p', 1), ('q', 1)), (('p', 2), ('p', 6)), (('z', 4), ('z', 1)), (('m', 2), ('m', 0))]}


In [2]:
# I'm thinking that I can enumerate "generator"x"applications" pairs
# like I would map from NN to QQ
def zig_zag(n):
    last = (0,0)
    idx = 0
    while idx < n:
        yield last
        idx += 1
        if last[1] == 0:
            last = (0,last[0]+1)
        else:
            last = (last[0]+1,last[1]-1)
    
list(zig_zag(10))

[(0, 0),
 (0, 1),
 (1, 0),
 (0, 2),
 (1, 1),
 (2, 0),
 (0, 3),
 (1, 2),
 (2, 1),
 (3, 0)]

In [3]:
# helper function needed to undo my zig zag enumeration
def triangular(n):
    return int((n+1)*n/2)

In [4]:
list(map(triangular,range(10)))

[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]

In [5]:
# reverse the zig zag enumeration
def reverse_zig_zag(p):
    my_sum = p[0]+p[1]
    my_triangle = triangular(my_sum)
    return my_triangle+p[0]

In [6]:
# test to make sure my inverse is working
list(map(reverse_zig_zag,list(zig_zag(10))))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
# now that we can enumerate pairs
# use that to standardize our relations
# want to always map from big numbers to small ones
def standardize_presentation(endomap):
    generator_map = {}
    reverse_generator_map = {}
    for idx,gen in enumerate(endomap['generators']):
        generator_map[idx] = gen
        reverse_generator_map[gen] = idx
    max_steps = 0
    relation_map = {}
    relation_pairs = {}
    max_depth = 0
    for rel in endomap['relations']:
        #print(f"Relation: {rel}")
        left_gen = reverse_generator_map[rel[0][0]]
        right_gen = reverse_generator_map[rel[1][0]]
        left_idx = reverse_zig_zag((left_gen,rel[0][1]))
        right_idx = reverse_zig_zag((right_gen,rel[1][1]))
        
        #print(f"L:{left_idx}, R:{right_idx}")
        my_min = min(left_idx,right_idx)
        my_max = max(left_idx,right_idx)
        if my_max > max_steps:
            max_steps = my_max
        relation_map[my_max] = my_min
        relation_pairs[left_idx] = rel[0]
        relation_pairs[right_idx] = rel[1]
        max_depth = max(max(rel[0][1],rel[1][1]),max_depth)
        
    return {
        "generator_count":len(endomap['generators']),
        "generators":endomap['generators'],
        "substitutions":relation_map,
        "relation_pairs":relation_pairs,
        "max_depth":max_depth,
        "max_steps":max_steps
    }
        

print(standardize_presentation(endomap_alpha))
print(standardize_presentation(endomap_beta))


    

{'generator_count': 4, 'generators': ['a', 'b', 'c', 'd'], 'substitutions': {15: 3, 4: 3, 8: 6, 18: 9}, 'relation_pairs': {15: ('a', 5), 3: ('a', 2), 4: ('b', 1), 6: ('a', 3), 8: ('c', 1), 18: ('d', 2), 9: ('d', 0)}, 'max_depth': 5, 'max_steps': 18}
{'generator_count': 4, 'generators': ['p', 'q', 'z', 'm'], 'substitutions': {4: 1, 21: 3, 23: 8, 18: 9}, 'relation_pairs': {1: ('p', 1), 4: ('q', 1), 3: ('p', 2), 21: ('p', 6), 23: ('z', 4), 8: ('z', 1), 18: ('m', 2), 9: ('m', 0)}, 'max_depth': 6, 'max_steps': 23}


In [8]:
def max_relation_steps(endomap):
    output = 0
    for r in endomap['relations']:
        output = max(output,max(r[0][1],r[1][1]))
    return output

max_relation_steps(endomap_alpha)

5

In [9]:
def standard_iterator(g,n):
    return "s"*n+g

def reconstruct_endomap(endomap):
    index_by_label = {}
    labels_by_index = {}
    my_map = {}
    #print(f"Attempting to enumerate points of {endomap}")
    my_pres = standardize_presentation(endomap)
    #print(f"Standardized presentation: {my_pres}")
    num_gens = len(endomap['generators'])
    max_steps = max_relation_steps(endomap)
    max_enum = reverse_zig_zag((num_gens,max_steps))
    #print(f"In order to get {max_steps} out of each generator, we need {max_enum} of zig zag")
    
    # make a map of known substitutions
    sub_map = {}
    for k,v in zip(my_pres['substitutions'].keys(),my_pres['substitutions'].values()):
        key_pair = my_pres['relation_pairs'][k]
        value_pair = my_pres['relation_pairs'][v]
        key_name = standard_iterator(key_pair[0],key_pair[1])
        value_name = standard_iterator(value_pair[0],value_pair[1])
        sub_map[key_name] = value_name
        
    #print(f"Sub Map: {sub_map}")
    for g_n in range(num_gens):
        #print(f"GENERATOR: {g_n}={endomap['generators'][g_n]}")
        a_n = 0
        my_pair = (g_n,a_n)
        my_zz_idx = reverse_zig_zag(my_pair)
        my_name = standard_iterator(endomap['generators'][g_n],a_n)
        #print(f"{my_pair} >> {my_zz_idx} = {my_name}")
        index_by_label[my_name] = my_zz_idx
        labels_by_index[my_zz_idx] = my_name
        
        while my_name not in my_map:
            a_n += 1
            #print(f"\tSTEP {a_n}") 
            my_pair = (g_n,a_n)
            my_zz_idx = reverse_zig_zag(my_pair)
            new_name = standard_iterator(endomap['generators'][g_n],a_n)
            #print(f"{my_pair} >> {my_zz_idx} = {new_name}")
            if new_name in sub_map:
                #print(f"\tFound sub for {new_name}!")
                new_name = sub_map[new_name]
                if new_name in index_by_label:
                    my_zz_idx = index_by_label[new_name]
            my_map[my_name] = new_name
            my_name = new_name
            index_by_label[my_name] = my_zz_idx
            labels_by_index[my_zz_idx] = my_name
            
    my_map_by_index = {index_by_label[k]:index_by_label[v] for k,v in zip(my_map.keys(),my_map.values())}        
    return {
        "map_by_name":my_map,
        "map_by_zz_index":my_map_by_index,
        "index_by_label":index_by_label,
        "label_by_index":labels_by_index,
        "size":len(my_map)
    }
    
reconstruct_endomap(endomap_alpha) 
    

{'map_by_name': {'a': 'sa',
  'sa': 'ssa',
  'ssa': 'sssa',
  'sssa': 'ssssa',
  'ssssa': 'ssa',
  'b': 'ssa',
  'c': 'sssa',
  'd': 'sd',
  'sd': 'd'},
 'map_by_zz_index': {0: 1, 1: 3, 3: 6, 6: 10, 10: 3, 2: 3, 5: 6, 9: 13, 13: 9},
 'index_by_label': {'a': 0,
  'sa': 1,
  'ssa': 3,
  'sssa': 6,
  'ssssa': 10,
  'b': 2,
  'c': 5,
  'd': 9,
  'sd': 13},
 'label_by_index': {0: 'a',
  1: 'sa',
  3: 'ssa',
  6: 'sssa',
  10: 'ssssa',
  2: 'b',
  5: 'c',
  9: 'd',
  13: 'sd'},
 'size': 9}

In [10]:
reconstruct_endomap(endomap_beta)

{'map_by_name': {'p': 'sp',
  'sp': 'ssp',
  'ssp': 'sssp',
  'sssp': 'ssssp',
  'ssssp': 'sssssp',
  'sssssp': 'ssp',
  'q': 'sp',
  'z': 'sz',
  'sz': 'ssz',
  'ssz': 'sssz',
  'sssz': 'sz',
  'm': 'sm',
  'sm': 'm'},
 'map_by_zz_index': {0: 1,
  1: 3,
  3: 6,
  6: 10,
  10: 15,
  15: 3,
  2: 1,
  5: 8,
  8: 12,
  12: 17,
  17: 8,
  9: 13,
  13: 9},
 'index_by_label': {'p': 0,
  'sp': 1,
  'ssp': 3,
  'sssp': 6,
  'ssssp': 10,
  'sssssp': 15,
  'q': 2,
  'z': 5,
  'sz': 8,
  'ssz': 12,
  'sssz': 17,
  'm': 9,
  'sm': 13},
 'label_by_index': {0: 'p',
  1: 'sp',
  3: 'ssp',
  6: 'sssp',
  10: 'ssssp',
  15: 'sssssp',
  2: 'q',
  5: 'z',
  8: 'sz',
  12: 'ssz',
  17: 'sssz',
  9: 'm',
  13: 'sm'},
 'size': 13}

In [11]:
# okay! now we can reconstruct the endomap from a presentation!
# now lets try decomposing each endomap into standardized "clusters"
def decompose_endomap(endomap):
    #print(f"Attempting to decompose {endomap}")
    my_pres = standardize_presentation(endomap)
    #print(f"Standardized presentation: {my_pres}")
    num_gens = len(endomap['generators'])
    max_steps = max_relation_steps(endomap)
    
    # go through our "substitutions" and collect cluster info
    clusters_by_generator = {}
    generators_by_cluster = {}
    for k in sorted(my_pres['substitutions'].keys()):
        v = my_pres['substitutions'][k]
        k_pair = my_pres['relation_pairs'][k]
        v_pair = my_pres['relation_pairs'][v]
        #print(f"{k_pair} {v_pair}")
        if k_pair[0] in clusters_by_generator:
            #print("left element already has a cluster")
            cluster_index = clusters_by_generator[k_pair[0]]
            if v_pair[0] not in generators_by_cluster[cluster_index]:
                generators_by_cluster[cluster_index].add(v_pair[0])
        elif v_pair[0] in clusters_by_generator:
            #print("right element already has a cluster")
            cluster_index = clusters_by_generator[v_pair[0]]
            if k_pair[0] not in generators_by_cluster[cluster_index]:
                generators_by_cluster[cluster_index].add(k_pair[0])
        else:
            cluster_index = len(generators_by_cluster)
            generators_by_cluster[cluster_index] = set([k_pair[0],v_pair[0]])
            clusters_by_generator[k_pair[0]] = cluster_index
            if v_pair[0] != k_pair[0]:
                clusters_by_generator[v_pair[0]] = cluster_index
    
    # having both of these maps, sort our relations into appropriate clusters
    clusters = []
    for value_set in generators_by_cluster.values():
        new_cluster = {
            'generators':list(value_set),
            'relations':list(filter(lambda x: x[0][0] in value_set,endomap['relations']))
        }
        clusters.append(new_cluster)
    return clusters

X_clusters = decompose_endomap(endomap_alpha)
X_clusters

[{'generators': ['a', 'b', 'c'],
  'relations': [(('a', 5), ('a', 2)),
   (('a', 2), ('b', 1)),
   (('a', 3), ('c', 1))]},
 {'generators': ['d'], 'relations': [(('d', 2), ('d', 0))]}]

In [12]:
Y_clusters = decompose_endomap(endomap_beta)
Y_clusters

[{'generators': ['p', 'q'],
  'relations': [(('p', 1), ('q', 1)), (('p', 2), ('p', 6))]},
 {'generators': ['m'], 'relations': [(('m', 2), ('m', 0))]},
 {'generators': ['z'], 'relations': [(('z', 4), ('z', 1))]}]

In [13]:
def get_loop_size(cluster):
    for rel in cluster['relations']:
        if rel[0][0] == rel[1][0]:
            return max(rel[0][1],rel[1][1])-min(rel[0][1],rel[1][1])
    return None

In [14]:
list(map(get_loop_size,X_clusters))

[3, 2]

In [15]:
list(map(get_loop_size,Y_clusters))

[4, 2, 3]

In [16]:
def find_all_cluster_pairs(endo_A,endo_B):
    # only clusters with the same cycle length could possibly match up
    clusters_A = decompose_endomap(endo_A)
    clusters_B = decompose_endomap(endo_B)
    pairs = []
    for cA in clusters_A:
        loopA = get_loop_size(cA)
        for cB in clusters_B:
            loopB = get_loop_size(cB)
            if loopA == loopB:
                #print(f"Potential for maps between {cA} and {cB}")
                pairs.append((cA,cB,loopA))
    return pairs
pairings = find_all_cluster_pairs(endomap_alpha,endomap_beta)
pairings

[({'generators': ['a', 'b', 'c'],
   'relations': [(('a', 5), ('a', 2)),
    (('a', 2), ('b', 1)),
    (('a', 3), ('c', 1))]},
  {'generators': ['z'], 'relations': [(('z', 4), ('z', 1))]},
  3),
 ({'generators': ['d'], 'relations': [(('d', 2), ('d', 0))]},
  {'generators': ['m'], 'relations': [(('m', 2), ('m', 0))]},
  2)]

In [17]:
lP = standardize_presentation(pairings[0][0])
lP

{'generator_count': 3,
 'generators': ['a', 'b', 'c'],
 'substitutions': {15: 3, 4: 3, 8: 6},
 'relation_pairs': {15: ('a', 5),
  3: ('a', 2),
  4: ('b', 1),
  6: ('a', 3),
  8: ('c', 1)},
 'max_depth': 5,
 'max_steps': 15}

In [18]:
rP = standardize_presentation(pairings[0][1])
rP

{'generator_count': 1,
 'generators': ['z'],
 'substitutions': {10: 1},
 'relation_pairs': {10: ('z', 4), 1: ('z', 1)},
 'max_depth': 4,
 'max_steps': 10}

In [19]:
X_clusters[1]

{'generators': ['d'], 'relations': [(('d', 2), ('d', 0))]}

In [20]:
XC = reconstruct_endomap(X_clusters[1])
XC

{'map_by_name': {'d': 'sd', 'sd': 'd'},
 'map_by_zz_index': {0: 1, 1: 0},
 'index_by_label': {'d': 0, 'sd': 1},
 'label_by_index': {0: 'd', 1: 'sd'},
 'size': 2}

In [21]:
YC = reconstruct_endomap(Y_clusters[1])
YC

{'map_by_name': {'m': 'sm', 'sm': 'm'},
 'map_by_zz_index': {0: 1, 1: 0},
 'index_by_label': {'m': 0, 'sm': 1},
 'label_by_index': {0: 'm', 1: 'sm'},
 'size': 2}

In [22]:
def make_assignments(gen,poss_vals):
  for v in poss_vals:
    yield {gen:v}

print(list(make_assignments("a",["x","y","z"])))

[{'a': 'x'}, {'a': 'y'}, {'a': 'z'}]


In [23]:
def make_map_by_assignment_vector(setA,setB,assignments):
    return {setA[i]:setB[assignments[i]] for i in range(len(setA))}

In [24]:
make_map_by_assignment_vector(['a','b','c'],['x','y'],[0,1,0])

{'a': 'x', 'b': 'y', 'c': 'x'}

In [25]:
def preserves_structure(f,x_map,y_map):
    for k,v in zip(f.keys(),f.values()):
        #print(f"f({k}) = {v}")
        alpha_x = x_map[k]
        f_alpha_x = f[alpha_x]
        #print(f"alpha({k}) = {alpha_x}")
        #print(f"f({alpha_x}) = {f_alpha_x}")
        beta_y = y_map[v]
        #print(f"beta({v}) = {beta_y}")
        if beta_y != f_alpha_x:
              #print("fails to preserve structure")
              return False
    return True

In [26]:
def make_all_maps(cA,cB):
    print(f"Cluster A: {cA}\nCluster B: {cB}")
    endo_A = reconstruct_endomap(cA)
    endo_B = reconstruct_endomap(cB)
    print(f"Endo A {endo_A}\nEndo B {endo_B}")
    
    total_maps = endo_B['size']**endo_A['size']
    print(f"Looking through {total_maps} maps")
    assignments = []
    idx = 0
    
    points_in_A = list(endo_A['map_by_name'].keys())
    size_A = len(points_in_A)
    points_in_B = list(endo_B['map_by_name'].keys())
    size_B = len(points_in_B)
    last_assignment = [0]*size_A
    
    while idx < total_maps:   
        if idx % 1000 == 0:
            print(f"Progress: {idx}/{total_maps}")
        new_map = make_map_by_assignment_vector(points_in_A,points_in_B,last_assignment)

        if preserves_structure(new_map,endo_A['map_by_name'],endo_B['map_by_name']):
            print(f"Found Structure Preserving Map! {new_map}")
            yield new_map

        idx += 1
        last_assignment[0] += 1
        for j in range(size_A-1):
            if last_assignment[j]>=size_B:
                last_assignment[j] = 0
                last_assignment[j+1] += 1
                
    
binary_maps = list(make_all_maps(X_clusters[1],Y_clusters[1]))
binary_maps

Cluster A: {'generators': ['d'], 'relations': [(('d', 2), ('d', 0))]}
Cluster B: {'generators': ['m'], 'relations': [(('m', 2), ('m', 0))]}
Endo A {'map_by_name': {'d': 'sd', 'sd': 'd'}, 'map_by_zz_index': {0: 1, 1: 0}, 'index_by_label': {'d': 0, 'sd': 1}, 'label_by_index': {0: 'd', 1: 'sd'}, 'size': 2}
Endo B {'map_by_name': {'m': 'sm', 'sm': 'm'}, 'map_by_zz_index': {0: 1, 1: 0}, 'index_by_label': {'m': 0, 'sm': 1}, 'label_by_index': {0: 'm', 1: 'sm'}, 'size': 2}
Looking through 4 maps
Progress: 0/4
Found Structure Preserving Map! {'d': 'sm', 'sd': 'm'}
Found Structure Preserving Map! {'d': 'm', 'sd': 'sm'}


[{'d': 'sm', 'sd': 'm'}, {'d': 'm', 'sd': 'sm'}]

In [27]:
list(make_all_maps(X_clusters[0],Y_clusters[2]))

Cluster A: {'generators': ['a', 'b', 'c'], 'relations': [(('a', 5), ('a', 2)), (('a', 2), ('b', 1)), (('a', 3), ('c', 1))]}
Cluster B: {'generators': ['z'], 'relations': [(('z', 4), ('z', 1))]}
Endo A {'map_by_name': {'a': 'sa', 'sa': 'ssa', 'ssa': 'sssa', 'sssa': 'ssssa', 'ssssa': 'ssa', 'b': 'ssa', 'c': 'sssa'}, 'map_by_zz_index': {0: 1, 1: 3, 3: 6, 6: 10, 10: 3, 2: 3, 5: 6}, 'index_by_label': {'a': 0, 'sa': 1, 'ssa': 3, 'sssa': 6, 'ssssa': 10, 'b': 2, 'c': 5}, 'label_by_index': {0: 'a', 1: 'sa', 3: 'ssa', 6: 'sssa', 10: 'ssssa', 2: 'b', 5: 'c'}, 'size': 7}
Endo B {'map_by_name': {'z': 'sz', 'sz': 'ssz', 'ssz': 'sssz', 'sssz': 'sz'}, 'map_by_zz_index': {0: 1, 1: 3, 3: 6, 6: 1}, 'index_by_label': {'z': 0, 'sz': 1, 'ssz': 3, 'sssz': 6}, 'label_by_index': {0: 'z', 1: 'sz', 3: 'ssz', 6: 'sssz'}, 'size': 4}
Looking through 16384 maps
Progress: 0/16384
Progress: 1000/16384
Progress: 2000/16384
Found Structure Preserving Map! {'a': 'sz', 'sa': 'ssz', 'ssa': 'sssz', 'sssa': 'sz', 'ssssa': 's

[{'a': 'sz',
  'sa': 'ssz',
  'ssa': 'sssz',
  'sssa': 'sz',
  'ssssa': 'ssz',
  'b': 'ssz',
  'c': 'z'},
 {'a': 'ssz',
  'sa': 'sssz',
  'ssa': 'sz',
  'sssa': 'ssz',
  'ssssa': 'sssz',
  'b': 'z',
  'c': 'sz'},
 {'a': 'ssz',
  'sa': 'sssz',
  'ssa': 'sz',
  'sssa': 'ssz',
  'ssssa': 'sssz',
  'b': 'sssz',
  'c': 'sz'},
 {'a': 'z',
  'sa': 'sz',
  'ssa': 'ssz',
  'sssa': 'sssz',
  'ssssa': 'sz',
  'b': 'sz',
  'c': 'ssz'},
 {'a': 'sssz',
  'sa': 'sz',
  'ssa': 'ssz',
  'sssa': 'sssz',
  'ssssa': 'sz',
  'b': 'sz',
  'c': 'ssz'},
 {'a': 'sz',
  'sa': 'ssz',
  'ssa': 'sssz',
  'sssa': 'sz',
  'ssssa': 'ssz',
  'b': 'ssz',
  'c': 'sssz'}]